1. Симуляция секвенирования (3 балла)
Первое задание заключается в том, чтобы написать симуляцию секвенирования ридов при помощи illumina. Для этого необходимо сгенерировать случайно строку длинной 50 000 символов, над алфавитом {A, T, G, C}. После этого случайно выбирать подстроки, длинна которых распределена нормально со средним 250 и среднеквадратическим отклонением 30, каждую такую подстроку "секвенировать".
Под словом "секвенировать" в данном случае имеется ввиду симуляция того процесса, который мы обсуждали на лекции. Вы считываете очередной символ подстроки, например, 100 раз, но в N случаях из 100 ошибаетесь и считываете любой нуклеотид, кроме правильного с одинаковой вероятностью. N принадлежит равномерному распределению от 0 до 60. По получившемуся набору вычисляете наибоее вероятный нуклеотид (тот, которого больше всего) и его качество прочтения, чтобы записать его в формате FASTQ.
При симуляции важно запоминать ID рида и для каждого ID позиции, в которых нуклеотид был считан неверно и какой должен быть на самом деле (для этого просто можно хранить 2 числа и помнить что у вас есть исходная строка, откуда берутся риды). Это понадобится для выполнения следующих заданий.
Всего ридов пусть будет 50К.

In [1]:
import random
import numpy as np
import json

In [ ]:
class Illumina:
    def __init__(self, seq_length, read_length_mean=250, read_length_std=30, error_rate_max=0.6):
        self.seq_length = seq_length
        self.read_length_mean = read_length_mean
        self.read_length_std = read_length_std
        self.error_rate_max = error_rate_max
        self.sequence = ''.join(random.choices(['A', 'T', 'G', 'C'], k=self.seq_length))
        self.reads = []
        self.errors = {}

    def simulate(self, n_reads):
        for i in range(n_reads):
            read_length = int(np.random.normal(loc=self.read_length_mean, scale=self.read_length_std))
            if read_length < 1:
                read_length = 1
            start_pos = random.randint(0, self.seq_length - read_length)
            read_seq = self.sequence[start_pos:start_pos + read_length]
            read_quality = ''.join([chr(random.randint(33, 73)) for _ in range(read_length)])
            errors = []
            for j in range(read_length):
                if random.random() < self.error_rate_max:
                    true_base = read_seq[j]
                    error_bases = [b for b in ['A', 'T', 'G', 'C'] if b != true_base]
                    error_base = random.choice(error_bases)
                    read_seq = read_seq[:j] + error_base + read_seq[j+1:]
                    errors.append((j, error_base, true_base))
            read_id = f"read_{i}"
            self.reads.append((read_id, read_seq, read_quality))
            if errors:
                self.errors[read_id] = errors

    def to_fastq(self, filename):
        with open(filename, 'w') as f:
            for read_id, read_seq, read_quality in self.reads:
                f.write(f"@{read_id}\n{read_seq}\n+\n{read_quality}\n")

    def to_json(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.errors, f)




In [ ]:

illumina = Illumina(seq_length=50000)
illumina.simulate(50000)
illumina.to_fastq('reads.fastq')
illumina.to_json('errors.json')

Получили два файла: reads.fastq и reads.json

2. Удаление ошибок (Trimmomatic) (2 балла) Получившиеся риды обработайте Trimmomatic со стандартными параметрами с лекции. Пользуясь тем, что вы знаете, для каждого рида ground truth, посчитайте, в скольких случаях Trimmomatic удалил нуклеотиды, которые были считаны верно, а в скольких действительно удалил ошибочные.

In [ ]:
def read_fastq(filename: str):
    sequences = []
    qualities = []
    with open(filename) as f:
        while True:
            f.readline()  # пропуск первой строки, содержащей информацию о риде
            seq = f.readline().rstrip()  # чтение последовательности рида
            f.readline()  # пропуск третьей строки, содержащей информацию о риде
            qual = f.readline().rstrip()  # чтение качества рида
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [ ]:
def read_json(filename: str):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

In [ ]:
# Чтение файлов
sequences, qualities = read_fastq('reads.fastq')
errors = read_json('errors.json')

In [ ]:
import subprocess

trim_command = ['java', '-jar', 'trimmomatic-0.39.jar', 'SE', '-phred33', '-', '-', 'LEADING:3', 'TRAILING:3',
                'SLIDINGWINDOW:4:15', 'MINLEN:36']

In [ ]:
correct_deleted = 0  # количество нуклеотидов, считанных верно и удаленных Trimmomatic
incorrect_deleted = 0  # количество нуклеотидов, считанных неверно и удаленных Trimmomatic

for i, seq in enumerate(sequences):
    # запуск Trimmomatic на каждом риде
    p = subprocess.Popen(trim_command, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = p.communicate(input=seq.encode())
    trimmed_seq = stdout.decode().strip()

    # определение количества удаленных нуклеотидов
    if trimmed_seq in seq:
        correct_deleted += len(seq) - len(trimmed_seq)
    else:
        for error_pos in errors[str(i)]:
            if error_pos < len(trimmed_seq) and seq[error_pos] != trimmed_seq[error_pos]:
                incorrect_deleted += 1
                break
        else:
            correct_deleted += len(seq) - len(trimmed_seq)

Получили, что с параметрами предложенными на лекции Trimmomatic просто все удаляет, чего, вообще говоря быть не должно

3. Коррекция ошибок (4 балла) Получившиеся риды обработайте одним(любым) из инструментов, которые использованы и проанализированны в этой статье. Посчитайте TP, FP, TN, FN.

Для подсчета TP, FP, TN, FN, необходимо иметь информацию о правильной классификации каждого рида (истинные метки) и о том, какие риды были отфильтрованы Trimmomatic (предсказанные метки). Для этого можно использовать информацию, сохраненную в JSON файле. Но так как Trimmomatic отработал вообще никак, то и посчитать данные характеристики не удалось.

Далее идет концепция кода, которая должна считать необходимые данные.

Для обработки хотелось использовать mpu

In [ ]:
import mpu

# Загрузка информации о неверно считанных нуклеотидах
with open('reads_errors.json', 'r') as f:
    reads_errors = json.load(f)

# Загрузка истинных меток ридов
# Для этого можно использовать информацию о том, какие подстроки были секвенированы
# и сравнить с исходной строкой
with open('ground_truth.txt', 'r') as f:
    ground_truth = f.read()

# Загрузка предсказанных меток из файла trimmed_reads.fastq
with open('trimmed_reads.fastq', 'r') as f:
    lines = f.readlines()
    # Отбрасываем строки, содержащие качество чтения
    reads = [lines[i] for i in range(len(lines)) if i % 4 == 1]

# Функция для преобразования ридов в список меток
def convert_reads_to_labels(reads):
    labels = []
    for read in reads:
        label = 1 if read in reads_errors else 0
        labels.append(label)
    return labels

# Преобразование истинных меток и предсказанных меток в списки
ground_truth_labels = convert_reads_to_labels([ground_truth[i:i+250] for i in range(0, len(ground_truth), 250)])
predicted_labels = convert_reads_to_labels(reads)

# Вычисление TP, FP, TN, FN
tp, fp, tn, fn = mpu.ml.confusion_matrix(ground_truth_labels, predicted_labels)

print('TP:', tp)
print('FP:', fp)
print('TN:', tn)
print('FN:', fn)


4. Сделайте какой-то вывод (1 балл)

То ли я что-то не так делаю, то ли Trimmomatic не умеет в исправление ошибок. Риды получили, а вот обработать их толком не получилось =(